In [235]:
import pandas as pd

In [236]:
df = pd.read_csv("./homicidios.csv")
df

,Regiao,Cidade,Mes,Ano,Total Crimes,Latitude,Longitude,populacao
0,Capital,São Paulo,Jan,2019,55.0,-23.533773,-46.625290,11509115.0
1,Capital,São Paulo,Fev,2019,48.0,-23.533773,-46.625290,11509115.0
2,Capital,São Paulo,Mar,2019,75.0,-23.533773,-46.625290,11509115.0
3,Capital,São Paulo,Abr,2019,59.0,-23.533773,-46.625290,11509115.0
4,Capital,São Paulo,Mai,2019,45.0,-23.533773,-46.625290,11509115.0
...,...,...,...,...,...,...,...,...
84439,Araçatuba,Santo Antônio do Aracanguá,Ago,2021,NaN,-21.207198,-50.449515,8364.0
84440,Araçatuba,Santo Antônio do Aracanguá,Set,2021,NaN,-21.207198,-50.449515,8364.0
84441,Araçatuba,Santo Antônio do Aracanguá,Out,2021,NaN,-21.207198,-50.449515,8364.0
84442,Araçatuba,Santo Antônio do Aracanguá,Nov,2021,NaN,-21.207198,-50.449515,8364.0


### ¿Cuál es la relación entre la ubicación geográfica y los homicidios?

In [237]:
print(df.columns)
print(df['Ano'].unique())
print(min(df['Ano'].unique()))
print(max(df['Ano'].unique()))

print("Cantidad Regiones: ", len(df['Regiao'].unique()))
print("Cantidad Ciudades: ", len(df['Cidade'].unique()))

print("Cantidad de Latitude", len(df['Latitude'].unique()))
print("Cantidad de Longitude", len(df['Longitude'].unique()))

print("Cantidad de Populacao", len(df['populacao'].unique()))

df['Total Crimes'] = df['Total Crimes'].fillna(0)


Index(['Regiao', 'Cidade', 'Mes', 'Ano', 'Total Crimes', 'Latitude',
       'Longitude', 'populacao'],
      dtype='object')
[2019 2018 2017 2016 2015 2014 2013 2012 2011 2010 2009 2008 2007 2006
 2005 2004 2003 2002 2001 2020 2021]
2001
2021
Cantidad Regiones:  11
Cantidad Ciudades:  594
Cantidad de Latitude 11
Cantidad de Longitude 11
Cantidad de Populacao 6630


In [238]:
center = df[['Latitude', 'Longitude']].mean().values.tolist()
ciudades_homicidios = df[['Cidade', 'Latitude', 'Longitude', 'Total Crimes']].dropna().groupby(['Cidade', 'Latitude', 'Longitude']).sum().reset_index()
total_homicidios = ciudades_homicidios['Total Crimes'].sum()

max_crimes = ciudades_homicidios['Total Crimes'].max()

ciudades_homicidios[ciudades_homicidios['Total Crimes'] == max_crimes]

,Cidade,Latitude,Longitude,Total Crimes
526,São Paulo,-23.533773,-46.62529,36597.0


In [239]:
import folium as fl
import numpy as np
from matplotlib import cm
from matplotlib.colors import Normalize

mapa = fl.Map(location=center, zoom_start=7.5)

min_crimes = ciudades_homicidios['Total Crimes'].min()
max_crimes = ciudades_homicidios['Total Crimes'].max()
norm = Normalize(vmin=min_crimes, vmax=max_crimes)

colormap = cm.get_cmap("turbo") 
ciudades_ordenadas = ciudades_homicidios.sort_values(by='Total Crimes')

/tmp/ipykernel_26349/2736198030.py:12: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap("turbo")


In [ ]:
for _, row in ciudades_ordenadas.iterrows():
    # Calculate radius based on log of crime count
    radius = np.log1p(row['Total Crimes']) * 5
    
    # Calculate color based on crime count
    crime_count = row['Total Crimes']
    color_value = colormap(norm(crime_count))
    
    # Convert matplotlib RGBA to hex color for folium
    hex_color = '#%02x%02x%02x' % (
        int(color_value[0] * 255),
        int(color_value[1] * 255),
        int(color_value[2] * 255)
    )
    
    # Crear el círculo con el número de homicidios como popup
    circle = fl.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=radius,
        color=hex_color,
        fill=True,
        fill_color=hex_color,
        fill_opacity=0.6,
        popup=f"{row['Cidade']}: {row['Total Crimes']} homicidios",
        tooltip=f"{row['Cidade']}: {row['Total Crimes']} homicidios"
    ).add_to(mapa)
    
    # Calcular tamaño de fuente basado en la cantidad de homicidios (para mejorar visibilidad)
    font_size = min(12 + np.log1p(row['Total Crimes'])/2, 18)  # Entre 12pt y 18pt
    
    # Ajustar la etiqueta para que los números más grandes sean más visibles
    fl.Marker(
        location=[row['Latitude'], row['Longitude']],
        icon=fl.DivIcon(
            icon_size=(150, 30),
            icon_anchor=(75, 15),
            html=f'''
                <div style="
                    background-color: rgba(255, 255, 255, 0.85);
                    padding: 3px 8px;
                    border-radius: 4px;
                    font-size: {font_size}pt;
                    font-weight: bold;
                    color: black;
                    text-align: center;
                    border: 1px solid {'#000' if row['Total Crimes'] > max_crimes/2 else '#666'};
                    box-shadow: 0 0 4px rgba(0,0,0,0.4);
                    display: inline-block;
                    z-index: {int(row['Total Crimes'])};  /* Valores más altos tendrán mayor z-index */
                ">
                    {int(row["Total Crimes"])}
                </div>
            '''
        )
    ).add_to(mapa)

# Add a legend
colormap_index = np.linspace(0, 1, 5)
legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; width: 150px; height: 120px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 10px">
    <p><b>Homicidios</b></p>
'''

for i in colormap_index:
    legend_color = colormap(i)
    hex_color = '#%02x%02x%02x' % (
        int(legend_color[0] * 255),
        int(legend_color[1] * 255),
        int(legend_color[2] * 255)
    )
    crime_value = int(min_crimes + i * (max_crimes - min_crimes))
    legend_html += f'<div style="background-color:{hex_color}; width:20px; height:15px; display:inline-block;"></div> {crime_value}<br>'

legend_html += '</div>'

mapa.get_root().html.add_child(fl.Element(legend_html))

mapa.save('index.html')